# Introduction

Dans ce notebook, on utilise le jeu de données issus de l'augmentation de données (ajout de taxons, API Plantnet...) afin de rechercher à nouveaux des motifs fréquents et potentielles règles d'associations intéressantes plantes-insectes.

Précédemment, nous avions recherché des motifs fréquents et règles d'associations importantes (lift supérieur à 1) plantes-insectes à partir des taxons présents dans la base du SPIPOLL : taxons _sc, _fr, _precision.

Pour un seuil de support minimal de 0.5, la recherche n'avait pas permis de mettre en évidence des motifs fréquents intéressants et règles d'associations importantes. L'abaissement du seuil à 0.4 avait permis de mettre en lumière un motif frelon-lierre, sans toutefois soulever une règle d'association importante. 

On se demande si d'autres motifs fréquents intéressants n'apparaitraient pas à un niveau de granularité plus global, grâce à l'approfondissement de la taxonomie. On va donc tester ici les niveaux de taxonomie suivant : 
- plante_famille, plante_genre, plante_espece
- insecte_ordre, insecte_super_famille, insecte_famille, insecte_sous_famille, insecte_genre, insecte_espece.


Dans le notebook précédent, nous avons testé pour la taxonomie la plus générale : plante_famille et insecte_ordre. Aucune conclusion biologiquement intéressante.

# Chargement des données

In [1]:
import pandas as pd

Décommentez la cellule suivante si vous avez besoin d'installer la librairie qui nous servira à la recherche de motifs fréquents.

In [2]:
# Installation de la librairie permettant la recherche de motifs fréquents 
#! pip install mlxtend

In [3]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [4]:
df = pd.read_csv("./spipoll.csv")

C:\Users\guilts0300\AppData\Local\Temp\ipykernel_23024\35506862.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./spipoll.csv")


In [5]:
unused_columns = ['collection_nom', 'collection_id', 'protocole_long', 'user_id',
       'photo_fleur', 'photo_plante', 'photo_feuille',               
       'plante_caractere', 'plante_inconnue',
       'photo_fleur', 'photo_plante', 'photo_feuille', 'coordonnees_GPS',
       'code_postal', 'fleur_ombre', 
       'insecte_abondance', 'insecte_commentaire', 'insecte_vu_sur_fleur',
       'nb_validation', 'nb_suggestion']

In [6]:
df.drop(columns=unused_columns, inplace=True)

In [7]:
# Transformation de la variable habitat

# One hot encoding après création de nouvelles colonnes pour chacun des habitats spécifiés entre ','
habitat_dummies = df['habitat'].str.get_dummies(',')

# Concaténation les colonnes de dummies avec le DataFrame d'origine
df = pd.concat([df, habitat_dummies], axis=1)

In [8]:
df.drop(columns=['habitat'], inplace=True)

In [9]:
df.columns

Index(['plante_famille', 'plante_genre', 'plante_espece', 'plante_sc',
       'plante_fr', 'plante_precision', 'grande_culture', 'collection_date',
       'collection_heure_debut', 'nebulosite', 'temperature', 'vent',
       'insecte_ordre', 'insecte_super_famille', 'insecte_famille',
       'insecte_sous_famille', 'insecte_genre', 'insecte_espece', 'insecte_sc',
       'insecte_fr', 'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref',
       'bord de l'eau', 'bord de route', 'forêt', 'grande(s) culture(s)',
       'jardin privé', 'littoral', 'parc ou jardin public', 'prairie',
       'péri-urbain', 'rochers', 'rural', 'urbain'],
      dtype='object')

# Recherche de motifs fréquents 

In [12]:
df['insecte_genre'].nunique()

333

In [14]:
df['insecte_genre'].value_counts()

Bombus          37429
Apis            30174
Lasioglossum    27995
Eristalis       22124
Calliphora      16863
                ...  
Setina              1
Danaus              1
Amata               1
Lepturobosca        1
Pseudoips           1
Name: insecte_genre, Length: 333, dtype: int64

In [15]:
df['plante_genre'].nunique()

871

In [16]:
df['plante_genre'].value_counts()

Heracleum        34343
Hedera           19843
Calendula        18789
Achillea         17881
Mentha           15167
                 ...  
Glycyrrhiza          1
Muehlenbeckia        1
Stipa                1
Lolium               1
Cupressus            1
Name: plante_genre, Length: 871, dtype: int64

Dans ce notebook, on utilise le jeu de données issus de l'augmentation de données (ajout de taxons, API Plantnet...) afin de rechercher à nouveaux des motifs fréquents et potentielles règles d'associations intéressantes plantes-insectes.

Précédemment, nous avions recherché des motifs fréquents et règles d'associations importantes (lift supérieur à 1) plantes-insectes à partir des taxons présents dans la base du SPIPOLL : taxons _sc, _fr, _precision.

Pour un seuil de support minimal de 0.5, la recherche n'avait pas permis de mettre en évidence des motifs fréquents intéressants et règles d'associations importantes. L'abaissement du seuil à 0.4 avait permis de mettre en lumière un motif frelon-lierre, sans toutefois soulever une règle d'association importante. 

On se demande si d'autres motifs fréquents intéressants n'apparaitraient pas à un niveau de granularité plus global, grâce à l'approfondissement de la taxonomie. On va donc tester ici les niveaux de taxonomie suivant : 
- plante_famille, plante_genre, plante_espece
- insecte_ordre, insecte_super_famille, insecte_famille, insecte_sous_famille, insecte_genre, insecte_espece.

## Vérification sur un exemple courant

On teste ici sur l'ordre incluant les bourdons, abeilles, frelons, etc. On aimerait trouver des motifs fréquents avec une famille de plantes.

In [17]:
data=df.where(df['insecte_genre']=='Bombus').dropna(subset=['insecte_genre'])

In [18]:
data.columns

Index(['plante_famille', 'plante_genre', 'plante_espece', 'plante_sc',
       'plante_fr', 'plante_precision', 'grande_culture', 'collection_date',
       'collection_heure_debut', 'nebulosite', 'temperature', 'vent',
       'insecte_ordre', 'insecte_super_famille', 'insecte_famille',
       'insecte_sous_famille', 'insecte_genre', 'insecte_espece', 'insecte_sc',
       'insecte_fr', 'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref',
       'bord de l'eau', 'bord de route', 'forêt', 'grande(s) culture(s)',
       'jardin privé', 'littoral', 'parc ou jardin public', 'prairie',
       'péri-urbain', 'rochers', 'rural', 'urbain'],
      dtype='object')

In [19]:
to_drop = ['plante_famille', 'plante_espece', 'plante_sc',
       'plante_fr', 'plante_precision', 'grande_culture',
       'collection_date', 'collection_heure_debut', 
        'insecte_super_famille',
       'insecte_famille', 'insecte_sous_famille', 'insecte_ordre',
       'insecte_espece', 'insecte_sc', 'insecte_fr',
       'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref']

In [20]:
data.drop(columns=to_drop, inplace=True)

In [21]:
data.columns

Index(['plante_genre', 'nebulosite', 'temperature', 'vent', 'insecte_genre',
       'bord de l'eau', 'bord de route', 'forêt', 'grande(s) culture(s)',
       'jardin privé', 'littoral', 'parc ou jardin public', 'prairie',
       'péri-urbain', 'rochers', 'rural', 'urbain'],
      dtype='object')

In [22]:
# One hot encoding
df_encoded = pd.get_dummies(data)
df_encoded = df_encoded.astype(bool)

In [23]:
# Recherche des motifs fréquents
frequent_itemsets = apriori(df_encoded, min_support=0.5, use_colnames=True)

# Filtrage des motifs fréquents de taille minimale égale à 2
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

# Classement par ordre décroissant de support 
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
frequent_itemsets

,support,itemsets
3,0.814769,"(nebulosite_0-25%, insecte_genre_Bombus)"
4,0.601539,"(temperature_20-30ºC, insecte_genre_Bombus)"


In [24]:
frequent_itemsets['itemsets'].unique()

array([frozenset({'nebulosite_0-25%', 'insecte_genre_Bombus'}),
       frozenset({'temperature_20-30ºC', 'insecte_genre_Bombus'})],
      dtype=object)

## Sur toute la base

In [32]:
# Suppression des lignes dont la valeur est manquante dans la colonne insecte_genre
df.dropna(subset=['insecte_genre'], inplace=True)

In [35]:
# Vérification
df['insecte_genre'].isnull().sum()

0

In [36]:
for insect in df['insecte_genre'].unique().tolist() :

    data=df.where(df['insecte_genre']==insect).dropna(subset=['insecte_genre'])
    
    #Nettoyage des colonnes inutiles (on ne garde que plante_famille pour les taxons de plantes)
    to_drop= ['plante_famille', 'plante_espece', 'plante_sc',
       'plante_fr', 'plante_precision', 'grande_culture',
       'collection_date', 'collection_heure_debut', 
        'insecte_super_famille',
       'insecte_famille', 'insecte_sous_famille', 'insecte_ordre',
       'insecte_espece', 'insecte_sc', 'insecte_fr',
       'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref']
    data.drop(columns=to_drop, inplace=True)
    
    # One hot encoding
    df_encoded = pd.get_dummies(data)
    df_encoded = df_encoded.astype(bool)
    
    # Algo a priori sur le jeu encodé
    frequent_itemsets = apriori(df_encoded, min_support=0.3, use_colnames=True)

    # Filtrer les motifs fréquents pour une taille minimale de 2
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

    # Classement par ordre décroissant de support 
    frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
    #print(frequent_itemsets)
    #print(frequent_itemsets['itemsets'].unique())
    
    # Recherche de règles d'association avec un lift supérieur à 1 
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1, support_only=True)
    
    # Affichage des règles d'association intéressantes
    if len(rules)>0: 
        print(insect, " : ", rules)

Callimus  :                  antecedents               consequents  antecedent support  \
0        (nebulosite_0-25%)  (insecte_genre_Callimus)                 NaN   
1  (insecte_genre_Callimus)        (nebulosite_0-25%)                 NaN   

   consequent support  support  confidence  lift  leverage  conviction  \
0                 NaN      1.0         NaN   NaN       NaN         NaN   
1                 NaN      1.0         NaN   NaN       NaN         NaN   

   zhangs_metric  
0            NaN  
1            NaN  
Toxophora  :                   antecedents                consequents  antecedent support  \
0  (insecte_genre_Toxophora)         (nebulosite_0-25%)                 NaN   
1         (nebulosite_0-25%)  (insecte_genre_Toxophora)                 NaN   

   consequent support  support  confidence  lift  leverage  conviction  \
0                 NaN      1.0         NaN   NaN       NaN         NaN   
1                 NaN      1.0         NaN   NaN       NaN         NaN   



Quelques règles d'associations importantes trouvées : 
- Lepturobosca  - plante_genre_Bistorta
- Lithosia  - plante_genre_Buddleja
- Amata   - plante_genre_Heracleum
- Danaus   - plante_genre_Buddleja
- Nudaria  - plante_genre_Galactites
- Setina  - plante_genre_Gentianella
- Deilephila  - plante_genre_Buddleja
- Laspeyria   - plante_genre_Calendula

## Interprétation

In [39]:
# Nombre d'observations des insectes en question qui sont dans une règle d'association importante
print(len(df.where(df['insecte_genre']=='Lepturobosca').dropna(subset=['insecte_genre'])))
print(len(df.where(df['insecte_genre']=='Lithosia').dropna(subset=['insecte_genre'])))
print(len(df.where(df['insecte_genre']=='Amata').dropna(subset=['insecte_genre'])))
print(len(df.where(df['insecte_genre']=='Danaus').dropna(subset=['insecte_genre'])))
print(len(df.where(df['insecte_genre']=='Nudaria').dropna(subset=['insecte_genre'])))
print(len(df.where(df['insecte_genre']=='Setina').dropna(subset=['insecte_genre'])))
print(len(df.where(df['insecte_genre']=='Deilephila').dropna(subset=['insecte_genre'])))
print(len(df.where(df['insecte_genre']=='Laspeyria').dropna(subset=['insecte_genre'])))

1
3
1
1
1
1
4
1


Ce sont des règles associations importantes car le support du motif fréquent est en fait égal à 1 et il n'y a pas d'autres observations venant infirmer la règle. 

## Conclusion

Pas de conclusion biologique pertinente. Règle d'association considérée comme importante dû à un nombre d'observations très faibles (inférieur à 5) pour les insectes en question.
On n'approfondira pas davantage une recherche large sur toute la base de données à différents niveaux de taxonomies.